# Problem 1 
## Detecting Poisonous Mushrooms 

This Problem has been solved in the following journal article using SOED: 

[Self-Organizing and Error Driven (SOED) artificial neural network for smarter](classifications[https://academic.oup.com/jcde/article/4/4/282/5729001)

We will be using ucimlrepo to import the sample data. Make sure to install it:

`pip install ucimlrepo`


In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 

  
# variable information 
print(mushroom.variables) 


                        name     role         type demographic  \
0                  poisonous   Target  Categorical        None   
1                  cap-shape  Feature  Categorical        None   
2                cap-surface  Feature  Categorical        None   
3                  cap-color  Feature       Binary        None   
4                    bruises  Feature  Categorical        None   
5                       odor  Feature  Categorical        None   
6            gill-attachment  Feature  Categorical        None   
7               gill-spacing  Feature  Categorical        None   
8                  gill-size  Feature  Categorical        None   
9                 gill-color  Feature  Categorical        None   
10               stalk-shape  Feature  Categorical        None   
11                stalk-root  Feature  Categorical        None   
12  stalk-surface-above-ring  Feature  Categorical        None   
13  stalk-surface-below-ring  Feature  Categorical        None   
14    stal

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from soed import SOEDClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

### Filling missing values

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cap-shape                 8124 non-null   object
 1   cap-surface               8124 non-null   object
 2   cap-color                 8124 non-null   object
 3   bruises                   8124 non-null   object
 4   odor                      8124 non-null   object
 5   gill-attachment           8124 non-null   object
 6   gill-spacing              8124 non-null   object
 7   gill-size                 8124 non-null   object
 8   gill-color                8124 non-null   object
 9   stalk-shape               8124 non-null   object
 10  stalk-root                5644 non-null   object
 11  stalk-surface-above-ring  8124 non-null   object
 12  stalk-surface-below-ring  8124 non-null   object
 13  stalk-color-above-ring    8124 non-null   object
 14  stalk-color-below-ring  

In [4]:
# Filling Missing Values
variable_df = mushroom.variables
for c,t in variable_df.set_index('name').drop(index=['poisonous']).type.to_dict().items():
    if t == 'Categorical':
        X.loc[:,c] = np.where(X[c].isna(),X[c].mode(),X[c])
    if t == 'Continuous':
        X.loc[:,c] = X[c].fillna(X[c].median())


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cap-shape                 8124 non-null   object
 1   cap-surface               8124 non-null   object
 2   cap-color                 8124 non-null   object
 3   bruises                   8124 non-null   object
 4   odor                      8124 non-null   object
 5   gill-attachment           8124 non-null   object
 6   gill-spacing              8124 non-null   object
 7   gill-size                 8124 non-null   object
 8   gill-color                8124 non-null   object
 9   stalk-shape               8124 non-null   object
 10  stalk-root                8124 non-null   object
 11  stalk-surface-above-ring  8124 non-null   object
 12  stalk-surface-below-ring  8124 non-null   object
 13  stalk-color-above-ring    8124 non-null   object
 14  stalk-color-below-ring  

### Binary Coding Categorical Attributes

In [6]:
X = pd.get_dummies(X)
dummy_vars = [c for c in X.columns if '_' in c]
X[dummy_vars] = X[dummy_vars].astype(int)

y = np.where(y=='p',1,0)[:,0]

In [7]:
dropping_columns = []
for c in X.columns:
    if X[c].std() == 0:
        print(c)
        dropping_columns.append(c)

veil-type_p


In [8]:
X = X.drop(columns = dropping_columns)

### Standardizing Data

In [9]:
X  = (X - X.mean())/X.std()

In [10]:
X

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,2.347113,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,-0.193076,-0.404817,4.590587,-0.155572
1,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.217810,-0.155572
2,4.119631,-0.022193,-0.796161,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,5.178669,-0.404817,-0.217810,-0.155572
3,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,-0.677492,1.226430,...,2.347113,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,-0.193076,-0.404817,4.590587,-0.155572
4,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.217810,-0.155572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,-0.242710,-0.022193,-0.796161,2.968250,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8120,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,1.005369,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8121,-0.242710,-0.022193,1.255873,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8122,-0.242710,-0.022193,-0.796161,2.968250,-0.062881,-0.904523,-0.632199,-0.022193,-0.677492,1.226430,...,-0.426003,1.005369,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572


### Split Data

In [11]:
random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.1))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X.iloc[train_index]
X_test = X.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

In [12]:
X_train

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
1003,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,1.005369,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
4444,-0.24271,-0.022193,1.255873,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,-0.677492,1.226430,...,-0.426003,-0.994537,1.935166,1.257176,-0.599494,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
6208,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,1.005369,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.21781,-0.155572
6631,-0.24271,-0.022193,1.255873,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,1.005369,-0.516688,1.257176,-0.599494,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
4640,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,-0.677492,1.226430,...,-0.426003,1.005369,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,1.581587,-0.022193,-0.677492,-0.815274,...,-0.426003,-0.994537,1.935166,1.257176,-0.599494,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
968,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,2.347113,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
1230,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,1.581587,-0.022193,-0.677492,-0.815274,...,2.347113,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572
4771,-0.24271,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,-0.677492,1.226430,...,-0.426003,-0.994537,1.935166,1.257176,-0.599494,-0.337763,-0.193076,-0.404817,-0.21781,-0.155572


### Model Trainig

In [13]:
soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train)

version 1.0.8
Model training complete.


### Model Evaluation

In [14]:
y_proba = soed.predict_proba(X_test.values)
y_pred = soed.predict(X_test.values)

recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
auc = roc_auc_score(y_test,y_proba[:,1])

performance = {'recall':recall,'precision':precision,'accuracy':accuracy,'auc':auc}

In [15]:
print(performance)

{'recall': 0.9934825729668462, 'precision': 0.9957398466344788, 'accuracy': 0.9948023526193407, 'auc': np.float64(0.9970943331366803)}


### Model Comparison

In [16]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','SOED'] )
report_df

,DT,SOED
repeat1,NaN,NaN
repeat2,NaN,NaN
repeat3,NaN,NaN
repeat4,NaN,NaN
repeat5,NaN,NaN


In [17]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.4))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000)
    soed.fit(X_train.values,y_train)
    y_proba = soed.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','SOED'] = auc

    #dt
    dt = DecisionTreeClassifier(max_depth=10)
    dt.fit(X_train.values,y_train)
    y_proba = dt.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','DT'] = auc


report_df.loc['Average'] = report_df.mean()

1
version 1.0.8
Model training complete.
2
version 1.0.8
Model training complete.
3
version 1.0.8
Model training complete.
4
version 1.0.8
Model training complete.
5
version 1.0.8
Model training complete.


In [18]:
report_df

,DT,SOED
repeat1,0.999146,1.0
repeat2,0.999579,1.0
repeat3,0.999363,0.999363
repeat4,1.0,0.999997
repeat5,0.99936,0.999004
Average,0.99949,0.999673


We can see that their performance is comparable. Let's see how they will do in the next problem.

# Problem 2
## Deciding to consume mushroom 

We must assume some data to use the mushroom dataset for a decision-making problem. We will assume that when we make a false negative, we will incur a 200-dollar mistake cost; when we make a false positive, we will incur a 5-dollar cost of errors.


In [19]:
c0 = np.where(y==0,0,200)
c1 = np.where(y==1,0,5)
c = np.column_stack((c0,c1))


random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.4))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X.iloc[train_index]
X_test = X.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

c_train = c[train_index,:]
c_test = c[test_index,:]

In [20]:
soed = SOEDClassifier(mlp_max_iter=10000)
soed.fit(X_train.values,y_train,c_train)

version 1.0.8
Model training complete.


In [21]:
y_decide = soed.decide(X_test.values)

In [22]:
df = pd.DataFrame(np.column_stack((y_test,y_decide)),columns = ['Reality','Decision'])
df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),200,0)
df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),5,0)
df['cost'] = df.FP_cost + df.FN_cost
df

,Reality,Decision,FN_cost,FP_cost,cost
0,0,0,0,0,0
1,1,1,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
4868,1,1,0,0,0
4869,0,0,0,0,0
4870,1,1,0,0,0
4871,0,0,0,0,0


In [23]:
total_cost = df.cost.sum()
print(f'total cost is {total_cost}.')

total cost is 2975.


In [24]:
df.cost.value_counts()

cost
0      4824
5        35
200      14
Name: count, dtype: int64

### Comparing Models

In [25]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','SOED'] )
report_df

,DT,SOED
repeat1,NaN,NaN
repeat2,NaN,NaN
repeat3,NaN,NaN
repeat4,NaN,NaN
repeat5,NaN,NaN


In [26]:
def calc_cost(y_reality,y_decide):
    df = pd.DataFrame(np.column_stack((y_reality,y_decide)),columns = ['Reality','Decision'])
    df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),200,0)
    df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),5,0)
    df['cost'] = df.FP_cost + df.FN_cost
    return df.cost.sum()

In [27]:
def get_cost_minimizing_threshold(y_reality,y_prob):
    candidate_df = pd.DataFrame(index=range(99),columns = ['thresh','cost'])
    candidate_df.thresh = np.linspace(0.01,0.99,99)
    candidate_df = candidate_df.set_index('thresh')
    
    for t in candidate_df.index.tolist():
        y_decide = np.where(y_prob>t,1,0)
        candidate_df.loc[t,'cost'] = calc_cost(y_reality,y_decide)
    return candidate_df[candidate_df.cost == candidate_df.cost.min()].index[0]

In [28]:
for loop_i in range(1,11):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.4))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    c_train = c[train_index,:]
    c_test = c[test_index,:]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000)
    soed.fit(X_train.values,y_train,c_train)
    y_decide = soed.decide(X_test.values)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','SOED'] = total_cost

    #dt
    dt = DecisionTreeClassifier()
    dt.fit(X_train.values,y_train)

    y_prob = dt.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob)
    print(f'dt threshod= {thresh}')
    y_prob = dt.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','DT'] = total_cost


report_df.loc['Average'] = report_df.mean()

1
version 1.0.8
Model training complete.
dt threshod= 0.01
2
version 1.0.8
Model training complete.
dt threshod= 0.01
3
version 1.0.8
Model training complete.
dt threshod= 0.01
4
version 1.0.8
Model training complete.
dt threshod= 0.01
5
version 1.0.8
Model training complete.
dt threshod= 0.01
6
version 1.0.8
Model training complete.
dt threshod= 0.01
7
version 1.0.8
Model training complete.
dt threshod= 0.01
8
version 1.0.8
Model training complete.
dt threshod= 0.01
9
version 1.0.8
Model training complete.
dt threshod= 0.01
10
version 1.0.8
Model training complete.
dt threshod= 0.01


In [29]:
report_df

,DT,SOED
repeat1,0,710
repeat2,0,5
repeat3,0,150
repeat4,800,725
repeat5,0,0
repeat6,80,210
repeat7,0,65
repeat8,1600,20
repeat9,0,105
repeat10,800,5


SOED outperforms DT.